# Arvak + Qiskit Integration

This notebook demonstrates seamless integration between Arvak and Qiskit, enabling:
- Bi-directional circuit conversion (Qiskit ↔ Arvak)
- Using Arvak as a Qiskit backend
- Comparing compilation strategies
- Leveraging Arvak's advanced compilation with Qiskit circuits

## Installation

```bash
pip install arvak[qiskit]
```

## Integration Check

First, let's verify that the Qiskit integration is available.

In [ ]:
import arvak

# Verify Qiskit integration is available
status = arvak.integration_status()
print("Available integrations:")
for name, info in status.items():
    icon = "✓" if info['available'] else "✗"
    print(f"  {icon} {name}: {info['packages']}")

# Check specifically for Qiskit
if 'qiskit' not in status or not status['qiskit']['available']:
    raise ImportError(
        "Qiskit integration not available. "
        "Install with: pip install qiskit>=1.0.0 qiskit-aer>=0.13.0"
    )

print("\n✓ Qiskit integration is available!")

## Create a Qiskit Circuit

Let's start with a standard Qiskit circuit.

In [ ]:
from qiskit import QuantumCircuit

# Create a Bell state circuit in Qiskit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure(range(2), range(2))

print("Qiskit Circuit:")
print(qc.draw('text'))

## Method 1: Direct Conversion via Integration API

In [ ]:
# Get Qiskit integration
qiskit_integration = arvak.get_integration('qiskit')

# Convert Qiskit circuit to Arvak
arvak_circuit = qiskit_integration.to_arvak(qc)

print("Arvak Circuit:")
print(f"  Name: {arvak_circuit.name}")
print(f"  Qubits: {arvak_circuit.num_qubits}")
print(f"  Classical bits: {arvak_circuit.num_clbits}")
print(f"  Depth: {arvak_circuit.depth()}")
print(f"  Gate count: {arvak_circuit.size()}")

## Method 2: Conversion via OpenQASM 3.0

You can also convert circuits using OpenQASM 3.0 as an interchange format. This works even without the Qiskit integration installed.

In [ ]:
from qiskit.qasm3 import dumps

# Export Qiskit circuit to QASM3
qasm_str = dumps(qc)
print("OpenQASM 3.0:")
print(qasm_str)
print("\n" + "="*60 + "\n")

# Import into Arvak
arvak_circuit_qasm = arvak.from_qasm(qasm_str)
print("Arvak Circuit (from QASM):")
print(f"  Qubits: {arvak_circuit_qasm.num_qubits}")
print(f"  Depth: {arvak_circuit_qasm.depth()}")

## Convert Arvak Circuit Back to Qiskit

In [ ]:
# Create a circuit in Arvak
arvak_original = arvak.Circuit.ghz(3)

# Convert to Qiskit
qiskit_circuit = qiskit_integration.from_arvak(arvak_original)

print("Converted to Qiskit:")
print(qiskit_circuit.draw('text'))

## Use Arvak as a Qiskit Backend

Arvak provides a Qiskit-compatible backend interface, allowing you to execute circuits through Qiskit's familiar API.

In [ ]:
from arvak.integrations.qiskit import ArvakProvider

# Get Arvak backends via Qiskit provider interface
provider = ArvakProvider()
print("Available Arvak backends:")
for backend in provider.backends():
    print(f"  - {backend.name}")

# Select simulator
backend = provider.get_backend('sim')
print(f"\nSelected backend: {backend.name}")
print(f"  Qubits: {backend.num_qubits}")
print(f"  Basis gates: {backend.basis_gates}")

## Execute Circuit on Arvak Backend

Runs on Arvak's built-in statevector simulator (up to ~20 qubits). The simulator executes entirely in Rust via PyO3 — no network calls, no mocks.

In [ ]:
# Execute using standard Qiskit API
job = backend.run(qc, shots=1000)
result = job.result()
counts = result.get_counts()

print("Execution Results:")
for bitstring, count in sorted(counts.items()):
    bar = "█" * int(count / 10)
    print(f"  {bitstring}: {count:4d} {bar}")

## Working with More Complex Circuits

In [ ]:
# Create a more complex Qiskit circuit (GHZ-3)
qiskit_complex = QuantumCircuit(3, 3)
qiskit_complex.h(0)
qiskit_complex.cx(0, 1)
qiskit_complex.cx(1, 2)
qiskit_complex.measure(range(3), range(3))

print("Original Qiskit circuit:")
print(qiskit_complex.draw('text'))
print()

In [ ]:
# Convert to Arvak
from qiskit.qasm3 import dumps
qasm_complex = dumps(qiskit_complex)
arvak_complex = arvak.from_qasm(qasm_complex)

print(f"Arvak GHZ-3 Circuit:")
print(f"  Depth: {arvak_complex.depth()}")
print(f"  Gates: {arvak_complex.size()}")
print(f"  Qubits: {arvak_complex.num_qubits}")
print("\nQASM representation:")
print(arvak.to_qasm(arvak_complex))

## Configure Compilation for Hardware

Arvak provides advanced compilation tools that can optimize circuits for specific hardware.

In [ ]:
from arvak import CouplingMap, BasisGates, PropertySet, Layout

# Configure for IQM hardware
coupling_map = CouplingMap.star(5)
basis_gates = BasisGates.iqm()
layout = Layout.trivial(5)

props = PropertySet().with_target(coupling_map, basis_gates)
props.set_layout(layout)

print("IQM Hardware Configuration:")
print(f"  Topology: star with 5 qubits")
print(f"  Edges: {coupling_map.edges()}")
print(f"  Native gates: {basis_gates.gates()}")

## Comparison: Different Backend Configurations

In [ ]:
# Compare different hardware configurations
backends_config = [
    ("IQM", BasisGates.iqm(), CouplingMap.star(5)),
    ("IBM", BasisGates.ibm(), CouplingMap.linear(5)),
    ("Simulator", BasisGates.universal(), CouplingMap.full(5))
]

print("Backend Comparison:")
print("=" * 70)
for name, gates, topology in backends_config:
    print(f"\n{name}:")
    print(f"  Native gates: {gates.gates()}")
    print(f"  Connectivity: {topology.edges()}")
    print(f"  Qubits: {topology.num_qubits}")

## Export for CLI Execution

Save your circuit for execution with the Arvak CLI.

In [ ]:
# Export circuit for CLI execution
output_qasm = arvak.to_qasm(arvak_complex)

# Save to file
with open("qiskit_circuit.qasm", "w") as f:
    f.write(output_qasm)

print("Circuit exported to qiskit_circuit.qasm")
print("\nTo execute:")
print("  $ arvak run qiskit_circuit.qasm --backend sim --shots 1000")
print("\nWith specific backend:")
print("  $ arvak run qiskit_circuit.qasm --backend iqm --shots 1000")

## Summary

This notebook demonstrated:

1. **Circuit Conversion**: Qiskit ↔ Arvak conversion via integration API or QASM
2. **Backend Interface**: Using Arvak as a Qiskit backend provider
3. **Compilation**: Configuring Arvak compilation for specific hardware
4. **Execution**: Running circuits through Qiskit's familiar API (or CLI for production)

## Next Steps

- Explore **03_qrisp_integration.ipynb** for high-level quantum programming
- Explore **04_cirq_integration.ipynb** for Google Cirq integration
- Use the Arvak CLI for production execution on real hardware
- Check out the Arvak dashboard for interactive circuit development

## Resources

- Arvak Documentation: https://github.com/hiq-lab/HIQ
- Qiskit Documentation: https://qiskit.org/documentation/
- OpenQASM 3.0 Spec: https://openqasm.com/